# Setup

In [37]:
%cd  /home/tlm/Work/FYP-22-23/weee-preprocess

/home/tlm/Work/FYP-22-23/weee-preprocess


In [38]:
import os
import datetime
import numpy as np
import pandas as pd
import neurokit2 as nk

In [39]:
DATASET_DIR = "./data/v1"
SAMPLING_RATE = 4 # Hz

# Prepare data

In [40]:
from utils.dataset import StudyInfoEncoder, DatasetVersion1
from utils.naming import standardize_column_names
from utils.resample import upsample
from utils.time import parse_simple_timedelta

In [41]:
dataset = DatasetVersion1()
activity_encoder = StudyInfoEncoder(os.path.join(DATASET_DIR, 'Study_Information.csv'))

# Process

In [42]:
def process_task(p):
    df = dataset.e4_eda(p)

    # process eda
    df = upsample(df, upsample_rate=SAMPLING_RATE*2, columns=['eda_raw'])
    signals, info = nk.eda_process(
        df['eda_raw'], sampling_rate=SAMPLING_RATE*2)
    df = pd.concat([df['timestamp'], signals], axis=1)

    # post touch ups
    df.insert(1, 'user_id', p)
    df = activity_encoder.fit_activity_column(
        df, p,
        timestamp_column='timestamp',
        activity_column='session_type',
        activity_column_index=2
    )
    df = standardize_column_names(df)
    print(f"Done processing participant {p:02d}")
    return df


In [43]:
# concat all participants row wise
E4_EDA = pd.concat([process_task(p) for p in range(1, 18)], axis=0, ignore_index=True)

Done processing participant 01
Done processing participant 02
Done processing participant 03
Done processing participant 04
Done processing participant 05
Done processing participant 06
Done processing participant 07
Done processing participant 08
Done processing participant 09
Done processing participant 10
Done processing participant 11
Done processing participant 12
Done processing participant 13
Done processing participant 14
Done processing participant 15
Done processing participant 16
Done processing participant 17


# Windowing

In [44]:
WINDOW_SIZES = ['2s', '4s', '6s', '8s', '10s', '12s']
AGG_FUNCS = ['mean', 'std', 'min', 'max', 'median']
SKIP_FROM_START_SIT = '30s'

In [45]:
def create_non_overlapping_windows(window_size, agg_funcs, ignore_incomplete_windows=True):
    dfs = []
    for p, p_df in E4_EDA.groupby('user_id'):
        
        if SKIP_FROM_START_SIT:
            skip = parse_simple_timedelta(SKIP_FROM_START_SIT)
            p_df = activity_encoder.crop_from_start_time(p_df, p, offset=skip)
        
        for a, pa_df in p_df.groupby('session_type'):
            # pre touch ups
            df = pa_df.sort_values('timestamp')

            # create windows and aggregate
            agg_cols = {c: agg_funcs for c in df.columns if c not in ['timestamp', 'session_type', 'user_id']}
            agg_cols['timestamp'] =  ['first', 'last']
            windowed = df.groupby(pd.Grouper(key='timestamp', freq=window_size, origin='start')).agg(agg_cols)

            # post touch ups
            windowed.columns = ['_'.join(col).strip() for col in windowed.columns.values]
            windowed = windowed.reset_index()
            windowed.insert(0, 'user_id', p)
            windowed.insert(1, 'session_type', a)
            windowed.insert(2, 'window_number', range(len(windowed)))
            windowed.insert(3, 'start_timestamp', windowed.pop('timestamp_first').dt.floor(freq='s'))
            windowed.insert(4, 'end_timestamp', windowed.pop('timestamp_last').dt.ceil(freq='s'))
            windowed.drop(columns=['timestamp'], inplace=True)
            if (windowed.iloc[-1]['end_timestamp'] - windowed.iloc[-1]['start_timestamp']) < parse_simple_timedelta(window_size):
                # remove last window if it is incomplete
                windowed.drop(windowed.tail(1).index, inplace=True)
                
            dfs.append(windowed)
            
    return pd.concat(dfs, axis=0, ignore_index=True)

In [46]:
%mkdir -p data/v2/E4
DATASET_DIR_V2 = "data/v2"

In [47]:
for ws in WINDOW_SIZES:
    E4_EDA_WINDOWED = create_non_overlapping_windows(ws, AGG_FUNCS)
    E4_EDA_WINDOWED.to_csv(os.path.join(DATASET_DIR_V2, 'E4', f"e4_eda_{ws}.csv"), index=False)
    print(f"Done processing window size {ws}")

Done processing window size 2s
Done processing window size 4s
Done processing window size 6s
Done processing window size 8s
Done processing window size 10s
Done processing window size 12s


In [48]:
E4_EDA_WINDOWED

,user_id,session_type,window_number,start_timestamp,end_timestamp,eda_raw_mean,eda_raw_std,eda_raw_min,eda_raw_max,eda_raw_median,...,scr_recovery_mean,scr_recovery_std,scr_recovery_min,scr_recovery_max,scr_recovery_median,scr_recovery_time_mean,scr_recovery_time_std,scr_recovery_time_min,scr_recovery_time_max,scr_recovery_time_median
0,1,0,0,2021-12-03 16:59:20,2021-12-03 16:59:32,1.545327,0.025702,1.507190,1.616096,1.537620,...,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0,1,2021-12-03 16:59:32,2021-12-03 16:59:44,1.481925,0.016698,1.453377,1.507190,1.482526,...,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,0,2,2021-12-03 16:59:44,2021-12-03 16:59:56,1.443881,0.020145,1.413658,1.489252,1.443447,...,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,0,3,2021-12-03 16:59:56,2021-12-03 17:00:08,1.433284,0.016772,1.403408,1.466190,1.431275,...,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,0,4,2021-12-03 17:00:08,2021-12-03 17:00:20,1.385030,0.010448,1.363689,1.404689,1.385471,...,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3283,17,5,41,2021-12-02 17:41:52,2021-12-02 17:42:04,1.930183,0.091294,1.777093,2.063804,1.944151,...,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3284,17,5,42,2021-12-02 17:42:04,2021-12-02 17:42:16,1.489226,0.182058,1.197126,1.771327,1.485287,...,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3285,17,5,43,2021-12-02 17:42:16,2021-12-02 17:42:28,1.037730,0.075662,0.949844,1.193282,1.013587,...,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3286,17,5,44,2021-12-02 17:42:28,2021-12-02 17:42:40,0.909586,0.042268,0.823541,0.970344,0.907464,...,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
